# EXAM 1

### Question 1a. - 1d.

See PDF

### Question 1e.

In this case, the antithetic sampling method should not be expect to reduce variance. This is due to the restricts on the
x values. This is due to the fact that the x values we use are pretty heavily skewed to one side of the uniform distribution.
The scale of that does not change when swapping to antithetic sampling in our case, so we don't see any positive impact on variance.
In fact, we see an increase in variance, but the difference is pretty minimal, so it's hard to tell if it's significant.

In other scenarios, Antithetic sampling will typically reduce variance by helping to normalize the sampled
values towards the mean of the distribution.

In our simulations the variance hovers between 0.0040 and 0.0046 for both uniform and antithetic sampling.

In [2]:
import numpy as np
import itertools


In [3]:
def integrateUniform(iterations):
    count = 0
    for _ in range(iterations):
        x = np.random.uniform(0, 1)
        # y is from 0 to 2
        y = np.random.uniform(0, 2)
        # b/c we are restricting to y values greater than 0,
        # we do not need to worry about when f(x) returns a value below 0
        if x < 0.5:
            count += (2 - 8 * (x ** 2)) >= y
    #(must divide by 1/2 )
    estimate = (count / iterations) * 2
    return estimate


def integrateAntithetic(iterations):
    count = 0
    for _ in range(iterations):
        x = np.random.uniform(0, 1)
        # y is from 0 to 2
        y = np.random.uniform(0, 2)
        ax = 1 - x
        if ax <= 0.5:
            count += (2 - 8 * (ax ** 2)) >= y
    estimate = (count / iterations) * 2
    return estimate

iters = 200
trials = 1000

print("Random Sampling")
uniform = [integrateUniform(iters) for _ in range(trials)]
print(f"Approx. Area:\t{np.mean(uniform):.2f}")
print(f"Variance: {np.var(uniform):.6f}\n")

print("Antithetic Sampling")
antithetic = [integrateAntithetic(iters) for _ in range(trials)]
print(f"Approx. Area:\t{np.mean(antithetic):.2f}")
print(f"Variance: {np.var(antithetic):.6f}\n")

Random Sampling
Approx. Area:	0.67
Variance: 0.004688

Antithetic Sampling
Approx. Area:	0.66
Variance: 0.004560



### Question 1f.
Equation on PDF

In [4]:
def integrateImportance(iterations):
    count = 0
    for _ in range(iterations):
        # b/c x > 0.5 ret 0 we can simplify to random 0,0.5 instead of 0,1
        x = np.random.uniform(0, 0.5)
        # y is from 0 to 2
        y = np.random.uniform(0, 2)
        count += (2 - 8 * (x ** 2)) >= y
    estimate = (count / iterations)
    return estimate


print("Importance Sampling")
importance = [integrateImportance(iters) for _ in range(trials)]
print(f"Approx. Area:\t{np.mean(importance):.2f}")
print(f"Variance: {np.var(importance):.6f}\n")


Importance Sampling
Approx. Area:	0.66
Variance: 0.001168



### 1g.
Importance sampling appears to be the absolute best in this case. This is primarily down to focusing our sampling to the region
of x's that actually have a chance at affecting our output (previously we had to discard many x values that had a 0 value).
By reducing the number of total samples, and focusing all samples on a smaller area, we see a substantially reduced
variance in our estimate. While the mean is unaffected, which is expected as the areas on the graph do not change,
the more precise sampling method means each iteration is contributing to our estimate in a more substantial way.

As a result, importance sampling is the best choice in this scenario.

# Question 2 Linear Programming

### Question 2a. Variables to Define

We need to define the following:

Cost of livestock setup

Cost of biohazard setup

Profit per livestock unit

Profit per biohazard unit

Indicator for Livestock Sold at Warehouse 1

Indicator for Livestock Sold at Warehouse 2

Indicator for Biohazard Sold at Warehouse 1

Indicator for Biohazard Sold at Warehouse 2

Total Shipping Hours Available at Warehouse 1

Total Shipping Hours Available at Warehouse 2

Number of Shipments Livestock at Warehouse 1

Number of Shipments Livestock at Warehouse 2

Number of Shipments Biohazard at Warehouse 1

Number of Shipments Biohazard at Warehouse 2

Big M for indicators

### Question 2b. Objective Function
Each warehouse has the same basic profit equation:

(Livestock per Hour * Livestock Profit * Livestock Shipments) +
(Biohazard per Hours * Biohazard Profit * Biohazard Shipments) +
(Livestock Indicator * -Livestock Setup Cost) +
(Biohazard Indicator * -Biohazard Setup Cost)


We want to maximize the profit of Warehouse 1 + Warehouse 2 as our objective function. This is fully expanded on the code below.

### Question 2c. Constraints


Indicator for Livestock Sold at Warehouse 1 {0,1}

Indicator for Livestock Sold at Warehouse 2 {0,1}

Indicator for Biohazard Sold at Warehouse 1 {0,1}

Indicator for Biohazard Sold at Warehouse 2 {0,1}

Number of Shipments Livestock at Warehouse 1 {0, inf}

Number of Shipments Livestock at Warehouse 2 {0, inf}

Number of Shipments Biohazard at Warehouse 1 {0, inf}

Number of Shipments Biohazard at Warehouse 2 {0, inf}

Warehouse 1 Livestock Shipments + Warehouse 1 Biohazard Shipments <= 240

Warehouse 2 Livestock Shipments + Warehouse 2 Biohazard Shipments <= 360

Warehouse 1 Livestock Shipments <= M * Warehouse 1 Livestock Sold Indicator

Warehouse 1 Biohazard Shipments <= M * Warehouse 1 Biohazard Sold Indicator

Warehouse 2 Livestock Shipments <= M * Warehouse 2 Livestock Sold Indicator

Warehouse 2 Biohazard Shipments <= M * Warehouse 2 Biohazard Sold Indicator

All variables are integers

We end up with a profit of $286200 selling 240 shipments of livestock at warehouse 1
and 360 shipments of livestock at warehouse 2

In [5]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

livestockCost = -22500
livestockProfit = 6
biohazardCost = -38000
biohazardProfit = 8
oneTotalShippingHours = 240
twoTotalShippingHours = 360

oneLivestockUnitsPerHour = 104
oneBiohazardUnitsPerHour = 76
twoLivestockUnitsPerHour = 84
twoBiohazardUnitsPerHour = 46

# w1Constraints = L + B <= 240
# w2Constraints = L + B <= 260
# w1Profit = (104 * 6 * L1) + (76 * 8 * B1) - 22500 * (Indicator L) - 38000 * (Indicator B)
# w2Profit = (84 * 6 * L2) + (46 * 8 * B2) - 22500 * (Indicator L) - 38000 * (Indicator B)
# Maximize = w1Profit + w2Profit

inf = solver.infinity()

oneLivestockIndicator = solver.IntVar(0, 1, 'Warehouse 1 Livestock Indicator')
twoLivestockIndicator = solver.IntVar(0, 1, 'Warehouse 2 Livestock Indicator')

oneBiohazardIndicator = solver.IntVar(0, 1, 'Warehouse 1 Biohazard Indicator')
twoBiohazardIndicator = solver.IntVar(0, 1, 'Warehouse 2 Biohazard Indicator')

oneLivestockShipments = solver.IntVar(0, inf, "Warehouse 1 Livestock")
twoLivestockShipments = solver.IntVar(0, inf, "Warehouse 2 Livestock")

oneBiohazardShipments = solver.IntVar(0, inf, "Warehouse 1 Biohazard")
twoBiohazardShipments = solver.IntVar(0, inf, "Warehouse 2 Biohazard")

m = 1800
solver.Add(oneLivestockShipments <= m * oneLivestockIndicator)
solver.Add(twoLivestockShipments <= m * twoLivestockIndicator)

solver.Add(oneBiohazardShipments <= m * oneBiohazardIndicator)
solver.Add(twoBiohazardIndicator <= m * twoBiohazardIndicator)

solver.Add(oneLivestockShipments + oneBiohazardShipments <= oneTotalShippingHours)
solver.Add(twoLivestockShipments + twoBiohazardShipments <= twoTotalShippingHours)

solver.Maximize(oneLivestockUnitsPerHour * livestockProfit * oneLivestockShipments + oneBiohazardUnitsPerHour * biohazardProfit * oneBiohazardShipments + (livestockCost * oneLivestockIndicator) + (biohazardCost * oneBiohazardIndicator) +
                twoLivestockUnitsPerHour * livestockProfit * twoLivestockShipments + twoBiohazardUnitsPerHour * biohazardProfit * twoBiohazardShipments + (livestockCost * twoLivestockIndicator) + (biohazardCost * twoBiohazardIndicator))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Maximum Profit =', solver.Objective().Value())
    print('Warehouse 1 Livestock =', oneLivestockShipments.solution_value())
    print('Warehouse 2 Livestock =', twoLivestockShipments.solution_value())
    print('Warehouse 1 Biohazard =', oneBiohazardShipments.solution_value())
    print('Warehouse 2 Biohazard =', twoBiohazardShipments.solution_value())
else:
    print('The problem does not have an optimal solution.')



Solution:
Maximum Profit = 286200.0
Warehouse 1 Livestock = 240.0
Warehouse 2 Livestock = 360.0
Warehouse 1 Biohazard = 0.0
Warehouse 2 Biohazard = 0.0


# Question 3

### Question 3a.

There are (52c5) ways to deal 5 cards from a 52 card deck which equates to 2,598,960


### Question 3b.

There are (52c5) * (47c5) ways to deal two five card hands from a standard deck of cards which equates to
3,986,646,103,440

### Question 3c.
There are 864 face pair flush hands.

(3c1) Face Cards * (4c2) Suits * (12c1) Non-Pair Cards *  (4c3) Suits

= 3 * 6 * 12 * 4

= 144 * 6

= 864

### Question 3d.
There are 864/(52c5) way sof being dealt a face pair flush, which equates to 864/2598960, or ~0.033%.

In [7]:
def buildDeck():
    suits = ['D', 'H', 'C', 'S']
    cards = range(1, 14)
    deck = []
    for c in cards:
        for j in suits:
            deck.append((c, j))
    return deck


def checkFullHouse(hand):
    cardValues = list(zip(*hand))[0]  # get values of each card in hand without their suit
    handCards = set(cardValues)  # get unique values of cards in hand
    faceCards = {11, 12, 13}  # create set of face cards we're interested in

    if len(handCards) == 2:  # check only 2 unique card values exist in hand
        if set(faceCards & set(handCards)):  # check we have at least 1 face card
            inter = handCards.intersection(faceCards)  # get value(s) of face card(s) the hand
            for i in inter:  # for each face card that exists in hand
                countInHand = cardValues.count(i)  # get count of face card in hand
                if countInHand == 2:  # if count is exactly 2, we have a pair of face cards and return a positive value
                    return 1
            return 0
        return 0
    return 0

fullDeck = buildDeck()
handSize = 5
iters = 0
x = 0
for hand in itertools.combinations(fullDeck, handSize): # runs through all 5 card hands
    x += checkFullHouse(hand)
    iters += 1

print(f"Total Face Pair Flush hands {x}\nTotal Hands Checked {iters}\nFace Pair Flush Probability {x / iters}")

Total Face Pair Flush hands 864
Total Hands Checked 2598960
Face Pair Flush Probability 0.0003324406685751224


# Question 4

### Question 4a.

I have liked the effort to teach better programming principles that are used int he real world, outside of just building code that works
even if it's ugly and hard to read.

I have disliked the reliance on python. I know it's used frequently, and that it is considered 'easy' to learn, but I find that
it tends to reinforce bad programming habits, and can often be difficult to adjust to when you're used to languages requiring a more
formal syntax.

### Question 4b.

I think the programming exercises that are lead by the professor/TA in class are the most helpful. Typically,
when we break into small groups it's one or two people that know how to solve the problem from the beginning and either
take over the conversation, or do not contribute much which can make it difficult to learn from.

### Question 4c.

I know it's probably not feasible, but I would have liked the opportunity to use any programming language to solve the problems.
At the very least, R is supported by Jupyter, as are .NET languages w/ .NET Core.
Since most assignments can be submitted as a jupyter notebook, I would have liked to have the option to use any language supported
by Jupyter. As python is my only major annoyance, I don't think much else needs to change. I've enjoyed the class so far.